In [44]:
from functools import wraps
from codetransformer.utils.pretty import a as show_st, d as show_disassembly

<center>
  <h1>Unspeakably Evil Hacks in Service of Marginally-Improved Syntax</h1><br>
  <h2>Compile-Time Metaprogramming in Python</h2><br>
  <h3>Scott Sanderson - <a href="https://github.com/ssanderson/pytenn2016">https://github.com/ssanderson/pytenn2016</a>
  </h3><br>
  
</center>

# Outline

- Standard Metaprogramming Tools
- Intro to CPython Compiler
- Import Hooks
- AST Transformers
- Bytecode Transformers

# Metaprogramming - Decorators

In [54]:
def print_inputs(f):
    "A decorator that prints inputs to a function before calling it."
    @wraps(f)
    def print_then_call_f(*args, **kwargs):
        print("Args: %s" % (args,))
        print("Kwargs: %s" % kwargs)
        f(*args, **kwargs)
    return print_then_call_f

In [53]:
@print_inputs
def my_func(a, b, c):
    print("Entering ``my_func``")
    print(a, b, c)
    print("Exiting ``my_func``")
    
my_func(1, 2, c=5)

Args: (1, 2)
Kwargs: {'c': 5}
Entering ``my_func``
1 2 5
Exiting ``my_func``


# Metaprogramming - Metaclasses

In [92]:
import inspect
import math

property_signature = inspect.FullArgSpec(
    args=['self'], 
    varargs=None, 
    varkw=None, 
    defaults=None, 
    kwonlyargs=[], 
    kwonlydefaults=None, 
    annotations={},
)


class AutoPropertyMeta(type):
    """
    A metaclass that wraps all no-argument methods of a subtype in properties.
    """
    def __new__(mcls, name, bases, clsdict):
        for name, class_attr in clsdict.items():
            try:
                signature = inspect.getfullargspec(class_attr)
            except TypeError: # Not everything in the classdict has to be callable.
                continue
            # Wrap anything with the right signature to be a property.
            if signature == property_signature:
                clsdict[name] = property(class_attr)

        return super().__new__(mcls, name, bases, clsdict)


In [ ]:
class Vector(metaclass=AutoPropertyMeta):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def magnitude(self):
        return math.sqrt(self.x ** 2 + self.y ** 2)
    
    def doubled(self):
        return Vector(self.x * 2, self.y * 2)

In [91]:
# Look mom, no parens!
print("Size: %s" % Vector(1, 2).magnitude)
print("Doubled Size: %s" % Vector(1, 2).doubled.magnitude)

Size: 2.23606797749979
Doubled Size: 4.47213595499958


# Metaprogramming - `exec`/`eval`

